In [1]:
#librerias
import requests #Para obtener links de internet
import re #Para utilizar Regex
import pandas as pd #Para generar Dataframes
import datetime
from datetime import date
import time
import re
import glob, os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Selenium

In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 958 kB 5.2 MB/s 
     |████████████████████████████████| 356 kB 49.1 MB/s 
     |████████████████████████████████| 138 kB 51.8 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 3.6 MB 36.7 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


# Lider

In [4]:
def Lider_data(link, producto):

    url = link + str(1000000) + '&page=1'
    print(url)
    driver.get(url)
    time.sleep(5)
    
    #Obtenemos el nombre del producto
    try:
      variedad = driver.find_elements_by_css_selector("span.product-name")
      variedad = [item.text for item in variedad]

      descr = driver.find_elements_by_css_selector("span.product-description.js-ellipsis")
      descr = [item.text for item in descr]

      atributos = driver.find_elements_by_css_selector("span.product-attribute")
      atributos = [item.text for item in atributos]

      precio = driver.find_elements_by_css_selector("span.price-sell")
      precio = [item.text for item in precio]
      print((len(descr)-2* len(atributos)+ len(precio))==0)

      data = pd.DataFrame({'Variedad': variedad,
                              'Descripcion': descr,
                              'Formato': atributos,
                              'Precio': precio})
      data.loc[:, 'Tienda'] = 'Lider'
      data.loc[:, 'Tipo Producto'] = producto
      data.loc[:, 'Fecha'] = datetime.datetime.today().strftime('%Y-%m-%d')


      data['Descripcion'] =  data['Descripcion'].str.lower()
      data['Formato'] =  data['Formato'].str.lower()
      data['Variedad'] =  data['Variedad'].str.lower()
      return data
    except:
      pass

# Tottus

In [5]:
def Tottus_data(link, pages, producto):
  df = {}
  for i in range(pages):
    url = link + str(i+1)
    print(url)
    driver.get(url)
    time.sleep(2)

    #Obtenemos el nombre del producto
    try:
      descr = driver.find_elements_by_css_selector("h2.jsx-28230554.name.title.mini")
      descr = [item.text for item in descr]

      atributos = driver.find_elements_by_css_selector("h3.jsx-28230554.extra.text.descriptive")
      atributos = [item.text for item in atributos]

      precio = driver.find_elements_by_css_selector("div.jsx-978899805.ProductPrice")
      if producto == "cerveza" or producto == "vinos":
          precio = [item.text for item in precio[4:]]
      else:
          precio = [item.text for item in precio]

      check = (len(descr)-2* len(atributos)+ len(precio))==0
      if not check:break
      if len(descr) == 0: break
      data = pd.DataFrame({'Variedad': 'na',
                              'Descripcion': descr,
                              'Formato': atributos,
                              'Precio': precio})
      data.loc[:, 'Tienda'] = 'Tottus'
      data.loc[:, 'Tipo Producto'] = producto
      data.loc[:, 'Fecha'] = datetime.datetime.today().strftime('%Y-%m-%d')

      data['Variedad'] =  data['Formato'].apply(lambda x: re.sub(' \- (.*)','', str(x))).str.lower()

      data['Descripcion'] =  data['Descripcion'].str.lower()

      data['Formato'] =  data['Formato'].apply(lambda x: re.sub('(.*) \- ','', str(x))).str.lower()
      data['Formato'] =  data['Formato'].apply(lambda x: re.sub('lt','l', str(x)))

      data['Precio'] =  data['Precio'].apply(lambda x: re.sub('UN','', str(x))).str.lower()
      
      df[i] = data
    except:
      pass
  return pd.concat(df.values(), axis=0, ignore_index=True)

# Scraper

In [6]:
links = {'bebidas':'https://www.tottus.cl/bebidas-cat010402/c/?sort=lowest_price&page=',
         'cerveza':'https://www.tottus.cl/cervezas-cat010403/c/?sort=Recomendados&page=',
         'vinos':'https://www.tottus.cl/vinos/c/?sort=Recomendados&page='}

data = {}
for i,j in zip(links.keys(), links.values()):
  data[i] = Tottus_data(j, 5, i)
df_tottus = pd.concat(data.values(), axis=0, ignore_index=True)

https://www.tottus.cl/bebidas-cat010402/c/?sort=lowest_price&page=1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


https://www.tottus.cl/bebidas-cat010402/c/?sort=lowest_price&page=2
https://www.tottus.cl/bebidas-cat010402/c/?sort=lowest_price&page=3
https://www.tottus.cl/bebidas-cat010402/c/?sort=lowest_price&page=4
https://www.tottus.cl/cervezas-cat010403/c/?sort=Recomendados&page=1
https://www.tottus.cl/cervezas-cat010403/c/?sort=Recomendados&page=2
https://www.tottus.cl/cervezas-cat010403/c/?sort=Recomendados&page=3
https://www.tottus.cl/cervezas-cat010403/c/?sort=Recomendados&page=4
https://www.tottus.cl/vinos/c/?sort=Recomendados&page=1
https://www.tottus.cl/vinos/c/?sort=Recomendados&page=2
https://www.tottus.cl/vinos/c/?sort=Recomendados&page=3
https://www.tottus.cl/vinos/c/?sort=Recomendados&page=4
https://www.tottus.cl/vinos/c/?sort=Recomendados&page=5


In [7]:
links = {'bebidas':'https://www.lider.cl/supermercado/category/Bebidas-Licores/Bebidas/_/N-o65v3z?No=0&isNavRequest=Yes&Nrpp=',
         'cerveza':'https://www.lider.cl/supermercado/category/Bebidas-Licores/Cervezas/_/N-1mi8n3m?No=40&isNavRequest=Yes&Nrpp=',
         'vinos':'https://www.lider.cl/supermercado/category/Bebidas-Licores/Vinos-y-Espumantes/_/N-she0ig?No=40&isNavRequest=Yes&Nrpp='}

data = {}
for i,j in zip(links.keys(), links.values()):
  data[i] = Lider_data(j, i)
df_lider = pd.concat(data.values(), axis=0, ignore_index=True)

https://www.lider.cl/supermercado/category/Bebidas-Licores/Bebidas/_/N-o65v3z?No=0&isNavRequest=Yes&Nrpp=1000000&page=1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


True
https://www.lider.cl/supermercado/category/Bebidas-Licores/Cervezas/_/N-1mi8n3m?No=40&isNavRequest=Yes&Nrpp=1000000&page=1
True
https://www.lider.cl/supermercado/category/Bebidas-Licores/Vinos-y-Espumantes/_/N-she0ig?No=40&isNavRequest=Yes&Nrpp=1000000&page=1
True


In [9]:
fecha = datetime.datetime.today().strftime('%Y-%m-%d')
fecha = "2021-12-17"
fecha

'2021-12-17'

In [15]:
df = pd.concat([df_lider, df_tottus], ignore_index=True, axis=0)
# df = pd.concat([df_lider], ignore_index=True, axis=0)
df.to_csv('/content/drive/MyDrive/IPC Bienes/Datos (Scraper)/BVC/BVC--'+fecha+'.csv', index=False)

In [13]:
df["Fecha"] = df["Fecha"].apply(lambda x: fecha)

In [14]:
df[(df['Tipo Producto'] == 'cerveza') & (df['Variedad'] == 'cristal')]

,Variedad,Descripcion,Formato,Precio,Tienda,Tipo Producto,Fecha
153,cristal,pack cerveza cero sin alcohol 6 botellas,6 botellas 355 ml c/u,$4.390,Lider,cerveza,2021-12-17
178,cristal,pack cerveza lager 6 latas,6 latas 350 cc c/u,$3.020,Lider,cerveza,2021-12-17
220,cristal,pack cerveza lager 24 latas,24 un,$7.890,Lider,cerveza,2021-12-17
1116,cristal,pack cerveza long neck,6 x 355cc,$4.150,Tottus,cerveza,2021-12-17
1120,cristal,pack cerveza cristal 12x350cc lata,pack x 12,$6.299,Tottus,cerveza,2021-12-17
1187,cristal,pack cerveza cristal 6x355cc,pack x 6,$4.969,Tottus,cerveza,2021-12-17
1188,cristal,pack cerveza cristal 6x355cc sin alcohol long ...,pack x 6,$4.490,Tottus,cerveza,2021-12-17
1202,cristal,pack cerveza cristal 6x350cc sin alcohol lata,pack x 6,$4.069,Tottus,cerveza,2021-12-17
1215,cristal,pack cerveza sin alcohol (jugo limã³n natural),6 x 355cc,$4.069,Tottus,cerveza,2021-12-17
1219,cristal,pack cerveza cristal 6x350cc lata,pack x 6,$3.519,Tottus,cerveza,2021-12-17
